In [4]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

def load_images_from_folder(folder):
    images = []
    labels = []
    for subfolder in ['Negative', 'Positive']:
        label = 0 if subfolder == 'Negative' else 1
        path = os.path.join(folder, subfolder)
        for filename in os.listdir(path):
            img = cv2.imread(os.path.join(path, filename))
            if img is not None:
                img = cv2.resize(img, (224, 224))
                images.append(img)
                labels.append(label)
    return np.array(images), np.array(labels)

train_images, train_labels = load_images_from_folder('/Users/niteshyadav/Lyme_Disease/train')
test_images, test_labels = load_images_from_folder('/Users/niteshyadav/Lyme_Disease/test')
val_images, val_labels = load_images_from_folder('/Users/niteshyadav/Lyme_Disease/val')

# Normalize images
train_images = train_images / 255.0
test_images = test_images / 255.0
val_images = val_images / 255.0


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def create_cnn_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model1 = create_cnn_model()
model2 = create_cnn_model()
model3 = create_cnn_model()



In [6]:
import random

def fitness(model, X, y):
    loss, accuracy = model.evaluate(X, y, verbose=0)
    return accuracy

def crossover(parent1, parent2):
    # Only crossover at the dense layers
    child = Sequential()
    for i, (layer1, layer2) in enumerate(zip(parent1.layers, parent2.layers)):
        if isinstance(layer1, Dense) and isinstance(layer2, Dense):
            if random.random() > 0.5:
                child.add(layer1)
            else:
                child.add(layer2)
        else:
            child.add(layer1)
    child.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return child

def mutate(model):
    for layer in model.layers:
        if isinstance(layer, Dense) and random.random() > 0.5:
            layer.units = random.randint(64, 256)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def genetic_algorithm(models, X, y, generations=10):
    population = models
    for generation in range(generations):
        population = sorted(population, key=lambda model: fitness(model, X, y), reverse=True)
        next_generation = population[:2]
        for _ in range(len(population) - 2):
            parent1, parent2 = random.sample(population[:10], 2)
            child = crossover(parent1, parent2)
            child = mutate(child)
            next_generation.append(child)
        population = next_generation
    return sorted(population, key=lambda model: fitness(model, X, y), reverse=True)[0]

best_model = genetic_algorithm([model1, model2, model3], train_images, train_labels)



In [8]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
import numpy as np

class StackingModel(BaseEstimator, ClassifierMixin):
    def __init__(self, base_models, meta_model):
        self.base_models = base_models
        self.meta_model = meta_model

    def fit(self, X, y):
        self.base_models_ = [list() for _ in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))

        for i, model in enumerate(self.base_models):
            kfold = KFold(n_splits=5, shuffle=True, random_state=1)
            for train_idx, holdout_idx in kfold.split(X, y):
                instance = self._clone_keras_model(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_idx], y[train_idx], epochs=5, verbose=0)
                y_pred = instance.predict(X[holdout_idx])[:, 0]
                out_of_fold_predictions[holdout_idx, i] = y_pred

        self.meta_model_.fit(out_of_fold_predictions, y)
        return self

    def _clone_keras_model(self, model):
        cloned_model = create_cnn_model()  # Ensure this function creates the same architecture as the input model
        cloned_model.set_weights(model.get_weights())
        return cloned_model

    def predict(self, X):
        meta_features = np.column_stack([
            np.mean([model.predict(X)[:, 0] for model in base_models], axis=0)
            for base_models in self.base_models_
        ])
        return self.meta_model_.predict(meta_features)

base_models = [best_model]
meta_model = LogisticRegression()
stacking_model = StackingModel(base_models=base_models, meta_model=meta_model)
stacking_model.fit(train_images, train_labels)




26/26 [==============================] - 3s 97ms/step


StackingModel(base_models=[<keras.src.engine.sequential.Sequential object at 0x1602f8820>],
              meta_model=LogisticRegression())

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Oversampling using ImageDataGenerator
datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True)

# Generate more positive samples
positive_images = train_images[train_labels == 1]
positive_labels = train_labels[train_labels == 1]
oversampled_images = []
oversampled_labels = []
for _ in range(len(train_images) - len(positive_images)):
    img, label = next(datagen.flow(positive_images, positive_labels, batch_size=1))
    oversampled_images.append(img[0])
    oversampled_labels.append(label[0])

# Combine with original dataset
train_images = np.concatenate((train_images, np.array(oversampled_images)))
train_labels = np.concatenate((train_labels, np.array(oversampled_labels)))

# Training with class weights
class_weights = {0: 1., 1: len(train_labels) / np.sum(train_labels)}
stacking_model.fit(train_images, train_labels, class_weight=class_weights)



In [ ]:
# Evaluate the final model on validation and test datasets
val_predictions = stacking_model.predict(val_images)
test_predictions = stacking_model.predict(test_images)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print("Validation Accuracy:", accuracy_score(val_labels, val_predictions))
print("Test Accuracy:", accuracy_score(test_labels, test_predictions))
print("Confusion Matrix:\n", confusion_matrix(test_labels, test_predictions))
print("Classification Report:\n", classification_report(test_labels, test_predictions))

